In [1]:
# -----------------------------------------
# read, write, input file
# -----------------------------------------

import pytest_shutil, shutil, os, pandas

# nsims
nsims = 5

# define root path for all folders
root_path = r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\input\swmm'
print(os.path.exists(root_path))

# read in lhs_sampled_params
lhs_design = pandas.read_csv(
    r'C:\Users\echelsvi\git\chelsvig_urban_pesticides\probabilistic_python\io\lhs_sampled_params.csv')

# round lhs decimals
lhs_design = lhs_design.round(
    {"NImperv": 4, "NPerv": 4, "SImperv":3, "SPerv": 3, "PctZero": 3, "MaxRate": 3, "MinRate": 3, "Decay": 2, "DryTime": 0,
     "Por": 3, "WP": 3, "FC": 3, "Ksat": 3, "Roughness": 4, "Kdecay": 3, "BCoeff2": 3, "WCoeff2": 3})
print(lhs_design.head())

Ite =1
# do the following for each simulation...
newfol = "input_" + str(Ite)
print(newfol)

newdir = os.path.join(root_path, newfol)
print(newdir)

if not os.path.exists(newdir):
    os.mkdir(newdir)
    print("Folder ", Ite, " created", "\n")
else:
    print("Folder ", Ite, "already exists")

os.getcwd()
os.chdir(newdir)

# copy base file into new file location
old_swmm5 = os.path.join(root_path, "NPlesantCreek.inp")
new_file = os.path.join(newdir, "NPlesantCreek.inp")
shutil.copyfile(old_swmm5, new_file)

# start reading the new file
new_swmm5 = open(new_file, "r")
filelines = new_swmm5.readlines()

 

True
   Unnamed: 0  NImperv   NPerv  SImperv  SPerv  PctZero  MaxRate  MinRate  \
0           0   0.0243  0.2176    1.663  2.694   40.044  119.901    0.356   
1           1   0.0152  0.3288    1.432  2.624   57.428   54.573    7.922   
2           2   0.0103  0.2024    1.411  4.672   42.669   14.351    9.796   
3           3   0.0237  0.4867    1.923  5.080    7.899   25.438    5.581   
4           4   0.0216  0.4077    1.594  2.852   57.356   34.045    2.325   

   Decay  DryTime    Por     WP     FC   Ksat  Roughness  Kdecay  BCoeff2  \
0   4.33      4.0  0.497  0.124  0.269  7.484     0.0197   0.047    1.959   
1   3.08     11.0  0.496  0.032  0.070  3.025     0.0136   0.159    1.417   
2   6.91     11.0  0.494  0.033  0.226  4.617     0.0101   0.085    1.606   
3   2.77     13.0  0.450  0.101  0.149  5.960     0.0140   0.006    1.426   
4   5.83      7.0  0.434  0.049  0.181  4.734     0.0135   0.086    0.794   

   WCoeff2  
0    0.525  
1    0.211  
2    0.860  
3    0.087  
4   

In [2]:
   # edit the new file
    # ---------------------------
    # parameter = NImperv
    # ---------------------------
Num = 113  # number of subcatchments
row_0 = 175
NImperv = lhs_design.loc[Ite - 1, "NImperv"]
print(NImperv)

i = 1
row_t = row_0 + (i - 1)
oldline = filelines[row_t]
print(type(oldline))
print(oldline)


0.0243
<class 'str'>
S1               0.013      0.15       2.54       5.08       17         OUTLET    



In [6]:
Num = 3  # number of subcatchments
row_0 = 175
NImperv = lhs_design.loc[Ite - 1, "NImperv"]
print(NImperv)

for i in range(1, Num + 1):
    row_t = row_0 + (i - 1)
    oldline = filelines[row_t]

    fixline = " ".join(oldline.split())
    newline = fixline.replace("0.013", str(NImperv))
    newline = newline + "\n"
    filelines[row_t] = newline

    # ---------------------------
    # parameter = NPerv
    # ---------------------------
Num = 3  # number of subcatchments
row_0 = 175
NPerv = lhs_design.loc[Ite - 1, "NPerv"]
print(NPerv)

for i in range(1, Num + 1):
    row_t = row_0 + (i - 1)
    oldline = filelines[row_t]

    newline = oldline.replace("0.15", str(NPerv))
    filelines[row_t] = newline
    
print(filelines[175:178])


0.0243
0.2176
['S1 0.0243 0.2176 2.54 5.08 17 OUTLET\n', 'S2 0.0243 0.2176 2.54 5.08 17 OUTLET\n', 'S3 0.0243 0.2176 2.54 5.08 17 OUTLET\n']
